# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [1]:
from __future__ import annotations

import csv
import sqlite3
from datetime import datetime

In [2]:
def as_int(x: str) -> int | None:
    try:
        return int(float(x))
    except ValueError:
        return None


def as_float(x: str) -> float | None:
    try:
        return float(x)
    except ValueError:
        return None


def as_timestamp(x: str, fmt='%Y-%m-%d') -> int:
    return int(datetime.strptime(x, fmt).timestamp())

In [7]:
con = sqlite3.connect('data/db/Chinook_Sqlite.sqlite')
cur = con.cursor()

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [8]:
stmt = """
select FirstName, LastName from Customer where Country == 'Canada';
"""
cur.execute(stmt)
cur.fetchall()

[('François', 'Tremblay'),
 ('Mark', 'Philips'),
 ('Jennifer', 'Peterson'),
 ('Robert', 'Brown'),
 ('Edward', 'Francis'),
 ('Martha', 'Silk'),
 ('Aaron', 'Mitchell'),
 ('Ellie', 'Sullivan')]

2. Найти и вывести на экран названия всех альбомов группы Accept

In [10]:
stmt = """
select Title from Album
where ArtistId = (select ArtistId from Artist where Name == 'Accept');
"""
cur.execute(stmt)
print(cur.fetchall())
con.close()

ProgrammingError: Cannot operate on a closed database.

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [11]:
con = sqlite3.connect('data/db/pi19_3.sqlite')
cur = con.cursor()

In [12]:
stmt = """
create table if not exists Student (
    id integer not null primary key,
    name varchar not null
)
"""
cur.execute(stmt)

In [13]:
stmt = """
insert into Student (name) values ('Alexey Kovalev');
"""
cur.execute(stmt)
con.commit()

In [14]:
stmt = """
select * from Student
"""
cur.execute(stmt)
students = cur.fetchall()
con.close()
students

[(1, 'Alexey Kovalev')]

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [15]:
con = sqlite3.connect('data/db/recipes.db')
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [16]:
stmt = """
drop table if exists Recipe
"""
cur.execute(stmt)

stmt = """
create table if not exists Recipe (
    id integer not null,
    name varchar,
    minutes real,
    submitted integer,
    description text,
    n_ingredients integer,
    constraint 'recipe_pk' primary key (id)
)
"""
cur.execute(stmt)

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [17]:
stmt = """
drop table if exists Review
"""
cur.execute(stmt)

stmt = """
create table if not exists Review (
    id integer not null,
    user_id integer,
    recipe_id integer,
    date integer,
    rating real,
    review text,
    constraint 'review_pk' primary key (id),
    constraint 'recipe_fk' foreign key (recipe_id) references Recipe(id)
)
"""
cur.execute(stmt)
con.commit()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [20]:
with open('data/csv/reviews_sample.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    cast = [
        as_int,
        as_int,
        as_int,
        as_timestamp,
        as_float,
        str
    ]
    data = [[cast[i](row[i]) for i in range(len(row))] for row in reader]

In [21]:
stmt = """
insert into Review (id, user_id, recipe_id, date, rating, review) values (?, ?, ?, ?, ?, ?)
"""
cur.executemany(stmt, data)
con.commit()

In [23]:
with open('data/csv/recipes_sample_with_filled_nsteps.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    index = [2, 1, 3, 5, 7, 8]
    cast = [as_int, str, as_float, as_timestamp, str, as_int]
    data = [[cast[j](row[i]) for j, i in enumerate(index)] for row in reader]

In [24]:
stmt = """
insert into Recipe (id, name, minutes, submitted, description, n_ingredients) values (?, ?, ?, ?, ?, ?)
"""
cur.executemany(stmt, data)
con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [25]:
stmt = """
select * from Recipe where n_ingredients == 10 limit 5;
"""
cur.execute(stmt)
cur.fetchall()

[(246, 'lee s hot crab dip', 45.0, 936133200, "lee's hot crab dip", 10),
 (289,
  'feijoada  brazilian bean soup  ii',
  150.0,
  938984400,
  'brazilian bean soup',
  10),
 (544, 'greek mushroom salad', 38.0, 934232400, '', 10),
 (671, 'cucumber relish', 40.0, 936910800, '', 10),
 (975,
  'basil parmesan biscuits',
  27.0,
  934578000,
  'perfect with pasta or other italian dishes',
  10)]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [26]:
stmt = """
select name from Recipe where minutes == (select max(minutes) from Recipe);
"""
cur.execute(stmt)
cur.fetchall()

[('strawberry liqueur',), ('blueberry liqueur',)]

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [27]:

def find_recipe_by_id():
    recipe_id = input('recipe_id: ')
    if not recipe_id.isdecimal():
        print('Not id')
        return

    recipe_id = int(recipe_id)
    stmt = """
    select * from Recipe where id == ?
    """
    cur.execute(stmt, [recipe_id])
    res = cur.fetchone()

    if not res:
        print('Not found')
        return

    return res

In [29]:
find_recipe_by_id()

(48, 'boston cream pie', 135.0, 935442000, '', 15)

In [30]:
find_recipe_by_id()

Not found


8. Найдите кол-во отзывов с рейтингом 5.

In [31]:
stmt = """
select count(*) from Review where rating == 5
"""
cur.execute(stmt)
cur.fetchone()

(91361,)

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [34]:
stmt = """
select count(distinct recipe_id) from Review where rating >= 4
"""
cur.execute(stmt)
cur.fetchone()

(26318,)

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [36]:
stmt = """
select count(id) from Recipe
where strftime('%Y', datetime(submitted, 'unixepoch')) == '2010' and minutes >= 15
"""
cur.execute(stmt)
cur.fetchone()

(1318,)

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [37]:
stmt = """
select REC.id, REC.name, REV.user_id, REV.date, REV.rating
from Recipe REC left join Review REV on REC.id = REV.recipe_id
where n_ingredients >= 3 order by REC.id;
"""
cur.execute(stmt)
cur.fetchall()


[(48, 'boston cream pie', 32421, 1016139600, 0.0),
 (48, 'boston cream pie', 68674, 1083531600, 2.0),
 (55, 'betty crocker s southwestern guacamole dip', 53959, 1137013200, 4.0),
 (55, 'betty crocker s southwestern guacamole dip', 165567, 1143752400, 5.0),
 (55, 'betty crocker s southwestern guacamole dip', 851190, 1274562000, 5.0),
 (55, 'betty crocker s southwestern guacamole dip', 1060485, 1239051600, 5.0),
 (66, 'black coffee barbecue sauce', 8679, 1047848400, 5.0),
 (66, 'black coffee barbecue sauce', 42938, 1035147600, 4.0),
 (66, 'black coffee barbecue sauce', 124416, 1278536400, 5.0),
 (66, 'black coffee barbecue sauce', 133174, 1278190800, 5.0),
 (66, 'black coffee barbecue sauce', 136004, 1212613200, 5.0),
 (66, 'black coffee barbecue sauce', 136813, 1176498000, 5.0),
 (66, 'black coffee barbecue sauce', 176798, 1285966800, 5.0),
 (66, 'black coffee barbecue sauce', 314792, 1202072400, 5.0),
 (66, 'black coffee barbecue sauce', 330545, 1230843600, 5.0),
 (66, 'black coffee ba